In [1]:
%load_ext autoreload
%autoreload 2

In [42]:
import os

while 'notebooks' in os.getcwd():
    os.chdir('..')

import pandas as pd
import torch
import torch.nn.functional as F
from torch_sparse.tensor import SparseTensor
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
from sklearn.metrics import roc_auc_score
import logging
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as pyo
import numpy as np

from src.torch_geo_models import GCN
from src.data.node_classifier.arxiv import load_dataset_pyg, data_to_sparse_symmetric_pyg

In [3]:
logging.basicConfig(
    format='%(asctime)s - %(levelname)s : %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S'
)

In [4]:
torch.cuda.is_available()

True

In [5]:
device = f'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
device

device(type='cuda', index=0)

## Data Loading

In [6]:
dataset = load_dataset_pyg()

In [17]:
data = data_to_sparse_symmetric_pyg(dataset[0])
data.adj_t = data.adj_t.to(device)

In [18]:
split_idx = dataset.get_idx_split()

In [19]:
split_idx

{'train': tensor([     0,      1,      2,  ..., 169145, 169148, 169251]),
 'valid': tensor([   349,    357,    366,  ..., 169185, 169261, 169296]),
 'test': tensor([   346,    398,    451,  ..., 169340, 169341, 169342])}

## Create `edge_weights`

In [14]:
edges_gamma_df = pd.read_csv('data/gamma_graph_sage/01-1_gamma_scored_edges.csv')
print(edges_gamma_df.shape)
edges_gamma_df.head()

(2315598, 3)


,source,target,gamma_pred
0,411,0,0.649669
1,640,0,0.784250
2,1162,0,0.787807
3,1897,0,0.787201
4,3396,0,0.784839


In [91]:
edge_weights = SparseTensor.from_torch_sparse_coo_tensor(
    torch.sparse_coo_tensor(
        edges_gamma_df[['source', 'target']].values.T,
        edges_gamma_df['gamma_pred'].values,
        size=(data.adj_t.size(0), data.adj_t.size(1))))\
    .to(device)

In [92]:
edge_weights.is_symmetric()

True

In [93]:
samp = edges_gamma_df.sample(1).iloc[0]
s, t = int(samp['source']), int(samp['target'])

(edge_weights[s, t], edge_weights[t, s])

(SparseTensor(row=tensor([0], device='cuda:0'),
              col=tensor([0], device='cuda:0'),
              val=tensor([0.7608], device='cuda:0', dtype=torch.float64),
              size=(1, 1), nnz=1, density=100.00%),
 SparseTensor(row=tensor([0], device='cuda:0'),
              col=tensor([0], device='cuda:0'),
              val=tensor([0.7608], device='cuda:0', dtype=torch.float64),
              size=(1, 1), nnz=1, density=100.00%))

### Model

In [94]:
features = data.x.cuda()
labels = data.y.cuda()
train_mask = split_idx['train'].cuda()
val_mask = split_idx['valid'].cuda()
test_mask = split_idx['test'].cuda()

In [95]:
train_nid = train_mask.nonzero().squeeze()
val_nid = val_mask.nonzero().squeeze()
test_nid = test_mask.nonzero().squeeze()

In [97]:
n_layers = 3
n_iters = 3000
epochs = 5000
log_steps = 100
input_dim = features.shape[1]
hidden_channels = input_dim * 2
output_dim = dataset.num_classes
lr_rate = 0.001
dropout = 0.5

In [98]:
model = GCN(
    n_layers=n_layers,
    in_channels=input_dim,
    hidden_channels=hidden_channels,
    out_channels=output_dim,
    dropout=dropout)\
    .to(device)

In [99]:
def train(model, graph, features, train_mask, optimizer, edge_weight=None):
    model.train()

    optimizer.zero_grad()
    out = model(features, graph, edge_weight=edge_weight)[train_mask]
    loss = F.nll_loss(out, labels.squeeze(1)[train_mask])
    loss.backward()
    optimizer.step()

    return loss.item()

In [100]:
@torch.no_grad()
def test(model, graph, features, labels, train_mask, val_mask, test_mask, evaluator, edge_weight=None):
    model.eval()

    out = model(features, graph, edge_weight=edge_weight)
    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': labels[train_mask],
        'y_pred': y_pred[train_mask],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': labels[val_mask],
        'y_pred': y_pred[val_mask],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': labels[test_mask],
        'y_pred': y_pred[test_mask],
    })['acc']

    return train_acc, valid_acc, test_acc

In [101]:
evaluator = Evaluator(name='ogbn-arxiv')

In [102]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr_rate)
for epoch in range(1, 1 + epochs):
    loss = train(model, data.adj_t, features, train_mask, optimizer, edge_weight=edge_weights)
    result = test(model, data.adj_t, features, labels, train_mask, val_mask, test_mask, evaluator, edge_weight=edge_weights)

    if epoch % log_steps == 0:
        train_acc, valid_acc, test_acc = result
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}, '
              f'Train: {100 * train_acc:.2f}%, '
              f'Valid: {100 * valid_acc:.2f}% '
              f'Test: {100 * test_acc:.2f}%')

Epoch: 100, Loss: 1.4663, Train: 62.37%, Valid: 63.15% Test: 62.02%
Epoch: 200, Loss: 1.1887, Train: 67.85%, Valid: 67.84% Test: 66.62%
Epoch: 300, Loss: 1.0999, Train: 69.66%, Valid: 69.17% Test: 67.88%
Epoch: 400, Loss: 1.0556, Train: 70.62%, Valid: 70.05% Test: 68.79%
Epoch: 500, Loss: 1.0204, Train: 71.30%, Valid: 70.66% Test: 69.24%
Epoch: 600, Loss: 0.9990, Train: 71.90%, Valid: 71.01% Test: 69.68%
Epoch: 700, Loss: 0.9783, Train: 72.33%, Valid: 71.21% Test: 70.05%
Epoch: 800, Loss: 0.9618, Train: 72.74%, Valid: 71.28% Test: 70.00%
Epoch: 900, Loss: 0.9442, Train: 73.12%, Valid: 71.54% Test: 70.34%
Epoch: 1000, Loss: 0.9295, Train: 73.44%, Valid: 71.65% Test: 70.72%
Epoch: 1100, Loss: 0.9143, Train: 73.79%, Valid: 71.80% Test: 70.66%
Epoch: 1200, Loss: 0.9025, Train: 74.09%, Valid: 71.76% Test: 70.64%
Epoch: 1300, Loss: 0.8930, Train: 74.38%, Valid: 72.09% Test: 71.05%
Epoch: 1400, Loss: 0.8828, Train: 74.68%, Valid: 71.93% Test: 70.71%
Epoch: 1500, Loss: 0.8711, Train: 74.90%, V